<a href="https://colab.research.google.com/github/123jyoti614/AI-Powered-Custom-Chatbot-For-PDF-Files/blob/main/AI_Powered_Custom_Chatbot_for_PDF_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 30.3 MB/s eta 0:00:00


In [ ]:

!pip install streamlit
!pip install langchain
!pip install sentence-transformers
!pip install faiss-cpu
!pip install -q streamlit
!pip install pyngrok
!pip install langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.1
    Uninstalling langchain-core-1.2.1:
      Successfully uninstalled langchain-core-1.2.1
ERROR: pip's dependency resol

In [ ]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings


In [ ]:
!ngrok authtoken 2tw0wKCaXZs4jvYdICTUycaUuhk_3Rnh7pgAP2bcsa642VHF1

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:

!ngrok

ngrok - tunnel local ports to public URLs and inspect traffic

USAGE:
  ngrok [command] [flags]

COMMANDS: 
  api             CLI to api.ngrok.com
  completion      generates shell completion code for bash or zsh
  config          update or migrate ngrok's configuration file
  credits         prints author and licensing information
  help            help about any command
  http            start an HTTP tunnel
  service         run and control ngrok as a background service
  start           start endpoints in the config file by name
  tcp             start a TCP tunnel
  tls             start a TLS endpoint
  update          update ngrok to the latest version
  version         print the version string

EXAMPLES: 
# forward http traffic from assigned public URL to local port 80
ngrok http 80
# port 8080 available at baz.ngrok.dev
ngrok http --url baz.ngrok.dev 8080
# tunnel arbitrary TCP traffic to port 22
ngrok tcp 22
# secure your app with oauth
ngrok http 80 --oauth=google --oauth-al

In [ ]:
%%writefile app.py
import streamlit as st
import os

import fitz  # PyMuPDF
import datetime as dt
from langchain.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub

# Function to load and process PDF
def load_pdf(uploaded_file):
    """Loads a PDF file and extracts its text content.

    Args:
        uploaded_file: The uploaded file object from Streamlit.

    Returns:
        A string containing the entire text content of the PDF.
    """
    pdf_text = []

    # Use uploaded_file.read() to get the file content
    with fitz.open(stream=uploaded_file.read(), filetype="pdf") as pdf:
        for page in pdf:
            pdf_text.append(page.get_text())
    return " ".join(pdf_text)

def process_text(pdf_text):
    texts = pdf_text.split(". ")
    grouped_texts, time_list = [], []
    for i, text in enumerate(texts):
        time_list.append(dt.datetime.now() + dt.timedelta(seconds=i*30))  # Set pseudo-timestamps for grouping
        grouped_texts.append(text)

    return grouped_texts, time_list

def get_vectorstore(grouped_texts, time_list):
    embeddings = HuggingFaceEmbeddings()
    time_stamps = [{'source': str(t)} for t in time_list]
    vectorstore = FAISS.from_texts(texts=grouped_texts, embedding=embeddings, metadatas=time_stamps)
    return vectorstore

def get_conversation(vectorstore):
    with open('token.txt', 'r') as f:
        token = f.read().strip()

    llm = HuggingFaceHub(
        repo_id="google/flan-t5-base",
        model_kwargs={"temperature": 0.5, "max_length": 512},
        huggingfacehub_api_token=token
    )
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        memory=memory,
        retriever=vectorstore.as_retriever()
    )
    return conversation_chain

def main():
    st.set_page_config(page_title="Chat with PDF", page_icon=":books:")
    st.header("Chat with your PDF Document 📄")

    uploaded_file = st.sidebar.file_uploader("Upload your PDF", type="pdf")

    if uploaded_file is not None:
        pdf_text = load_pdf(uploaded_file)
        grouped_texts, time_list = process_text(pdf_text)
        vectorstore = get_vectorstore(grouped_texts, time_list)

        st.write("Now you can start chatting!")

        user_question = st.text_input("Enter your question here:")
        if user_question:
            chain = get_conversation(vectorstore)
            response = chain({"question": user_question, "chat_history": []})['answer']
            st.write(response)

if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:

!streamlit run app.py &>/dev/null&

In [ ]:

!pgrep streamlit



1207
2220
3042


In [ ]:
!pip install pyngrok
from pyngrok import ngrok

# Attempt to kill any existing ngrok processes
try:
    ngrok.kill()
except Exception as e:
    print(f"Error killing existing process: {e}")
    print("Attempting to kill manually...")
    !pkill -f ngrok  # Manually kill any process with "ngrok" in its name

# Now you can connect to port 8501
public_url = ngrok.connect(8501)
public_url

<NgrokTunnel: "https://a738e2498d6f.ngrok-free.app" -> "http://localhost:8501">